In [3]:
import torch

In [11]:
x = torch.rand((10, 5))
labels = torch.randint(0,6,size=(10,))

In [12]:
labels

tensor([2, 3, 3, 5, 5, 4, 3, 0, 1, 2])

In [13]:
x

tensor([[0.0525, 0.4710, 0.6609, 0.9020, 0.8741],
        [0.4473, 0.3561, 0.1850, 0.6548, 0.3493],
        [0.6517, 0.4642, 0.9187, 0.5817, 0.8642],
        [0.4555, 0.3273, 0.6956, 0.2482, 0.9972],
        [0.0563, 0.9447, 0.4498, 0.7288, 0.9153],
        [0.5834, 0.0771, 0.7608, 0.2570, 0.3692],
        [0.8934, 0.5532, 0.3962, 0.3744, 0.1055],
        [0.2896, 0.4702, 0.7483, 0.8952, 0.6533],
        [0.3158, 0.6109, 0.9765, 0.1819, 0.2682],
        [0.0892, 0.8559, 0.2724, 0.4401, 0.1413]])

In [14]:
loss_known = torch.nn.CrossEntropyLoss()(x[labels != 5], labels[labels!=5])
loss_unknown = torch.nn.CrossEntropyLoss()(x[labels==5], x[labels==5, :4].argmax(axis=1))

lmbda = 1.
# loss = self.loss_binary(x_binary.flatten(), labels[:, 0].float()) + lmbda * self.loss_categ(x_categ[~zero_cat], labels[~zero_cat, 1])

loss = loss_known + lmbda * loss_unknown


In [17]:
x[labels != 5], labels[labels!=5]

(tensor([[0.0525, 0.4710, 0.6609, 0.9020, 0.8741],
         [0.4473, 0.3561, 0.1850, 0.6548, 0.3493],
         [0.6517, 0.4642, 0.9187, 0.5817, 0.8642],
         [0.5834, 0.0771, 0.7608, 0.2570, 0.3692],
         [0.8934, 0.5532, 0.3962, 0.3744, 0.1055],
         [0.2896, 0.4702, 0.7483, 0.8952, 0.6533],
         [0.3158, 0.6109, 0.9765, 0.1819, 0.2682],
         [0.0892, 0.8559, 0.2724, 0.4401, 0.1413]]),
 tensor([2, 3, 3, 4, 3, 0, 1, 2]))

In [18]:
x[labels==5], x[labels==5, :4].argmax(axis=1)

(tensor([[0.4555, 0.3273, 0.6956, 0.2482, 0.9972],
         [0.0563, 0.9447, 0.4498, 0.7288, 0.9153]]),
 tensor([2, 1]))

In [1]:
pip install -U transformers datasets evaluate

  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Using cached datasets-2.12.0-py3-none-any.whl (474 kB)
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached multiprocess-0.70.14-py310-none-any.whl (134 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached xxhash-3.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached pyarrow-12.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.9 MB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached yarl-1.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (268 kB)
  Using cached multidict-6.0.4-cp310-cp310

In [1]:
import pandas as pd

In [2]:
df_A = pd.read_csv("subtaskA_train.csv", index_col=0)
df_test_A = pd.read_csv("subtaskA_test.csv", index_col=0)

df_B = pd.read_csv("subtaskB_train.csv", index_col=0).drop(columns=["topic"])
df_test_B = pd.read_csv("subtaskB_test.csv", index_col=0)

df = df_A.merge(df_B, on="comment_text", how="outer")

df.loc[~df["conspiracy"].isna(), "conspiratorial"] = 1
df.loc[df["conspiratorial"] == 0, "conspiracy"] = -1
df.loc[df["conspiracy"].isna(), "conspiracy"] = -1

In [5]:
from transformers import AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("morenolq/bart-it")

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
import torch

In [9]:
from tfidf import load_vocabs, get_vec

In [10]:
vocabs_tfidf, mat_tfidf = load_vocabs()

### Encode with transformer's tokenizer

In [149]:
X = tokenizer(df.comment_text.tolist(), max_length=200, padding=True, return_tensors="pt", truncation=True)["input_ids"].cuda()
n_tokens = 52000
# X = X[y[:,1] != -1]
# y = y[y[:,1] != -1]

# X_test = tokenizer(df_test.comment_text.tolist(), max_length=200, padding=True, return_tensors="pt", truncation=True)["input_ids"]
# y_test = torch.tensor(df_test.conspiratorial.values).float()

### Encode with nltk

In [84]:
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from unidecode import unidecode

def encode_df(df, tokens_map, max_len = None):
    tokenized = []
    for sent in df.comment_text:
        toks = tokenizer.tokenize(unidecode(sent).lower())
        tokenized.append([ tokens_map.get(tok, len(tokens_map)) for tok in toks ])
    
    if max_len is not None:
        max_len = max_len if max_len != "compute" else max(map(len, tokenized))
        tokenized = [ tokens[:max_len] + [len(tokens_map)] * (max_len - len(tokens)) for tokens in tokenized ]
    return tokenized


tokenizer = RegexpTokenizer(r'\w+')

tok_freq = {}
for sent in df.comment_text:
    toks = tokenizer.tokenize(unidecode(sent).lower())
    for tok in toks:
        tok_freq[tok] = tok_freq.get(tok, 0) + 1
        
threshold = 0# 0 => take all!
# tokens = { k: v for k, v in tok_freq.items() if v > threshold }
tokens = sorted([ k for k, v in tok_freq.items() if v > threshold ])
tokens_map = { k: v for v, k in enumerate(tokens) }

n_tokens = len(tokens_map)

X = torch.tensor(encode_df(df, tokens_map, max_len="compute")).cuda()


In [85]:
y = torch.tensor(df[["conspiratorial", "conspiracy"]].values).float().cuda()
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=.8, random_state=42)

In [86]:
import numpy as np

In [87]:
mat_info = np.vstack([ get_vec(t, vocabs_tfidf, mat_tfidf) for t in tokens ])

In [88]:
import torch 
import torch.nn as nn

class ConvModel(nn.Module):
    def __init__(self, n_tokens=52000, extra_vecs=None):
        super().__init__()
        emb_size = 768
        conv1_channels = 64
        proj_size = 16
        
        if n_tokens == 52000:
            # assuming bart-it used, use padding_idx = 1 (filler token)
            padding_idx = 1
        else:
            # assuming custom tokenizer, use padding_idx = n_tokens + 1 for the filler
            padding_idx = n_tokens
            n_tokens += 1
        
        self.emb = nn.Embedding(num_embeddings=n_tokens, embedding_dim=emb_size, padding_idx=padding_idx)
        
        if extra_vecs is not None:
            # make sure we are passing a matrix shaped correctly
            # (+1 to account for the +1 that is being passed when n_tokens != 52000
            # NOTE: we do not expect to use this stuff when using pretrained vectors!
            # (b/c they tokenize things weirdly and it's hard to have a 1:1 mapping w/ tfidf
            assert n_tokens == extra_vecs.shape[0] + 1
            self.extra_emb = nn.Embedding(num_embeddings=n_tokens, embedding_dim=extra_vecs.shape[1], padding_idx=padding_idx)
            with torch.no_grad():
                self.extra_emb.weight[:-1] = torch.tensor(extra_vecs)
            
            emb_size += extra_vecs.shape[1]
        else:
            self.extra_emb = None
        
        self.conv1 = nn.Conv1d(emb_size, conv1_channels, stride=1, kernel_size=3)
        self.conv2 = nn.Conv1d(conv1_channels, 32, stride=1, kernel_size=5)
        self.conv3 = nn.Conv1d(32, 8, stride=3, kernel_size=5)
        
        self.mp = nn.MaxPool1d(kernel_size=3, stride=3)
        
        self.fc1 = nn.Linear(56, proj_size)
        self.head_bin = nn.Linear(proj_size, 1)
        self.head_cat = nn.Linear(proj_size, 4)
        
    def forward(self, x):
        x1 = self.emb(x).permute(0, 2, 1)
        if self.extra_emb is not None:
            x = torch.cat([ x1, self.extra_emb(x).permute(0,2,1) ], axis=1)
        else:
            x = x1
        x = torch.relu(self.mp(self.conv1(x)))
        x = torch.relu(self.mp(self.conv2(x)))
        x = torch.relu(self.mp(self.conv3(x)))
        
        x = x.reshape(x.shape[0], -1)
        
        x = torch.relu(self.fc1(x))
        x_bin = self.head_bin(x)
        x_cat = self.head_cat(x)
        
        return x_bin.flatten(), x_cat

In [90]:
import torch.optim as optim

In [91]:
def bce_xent():
    bce = nn.BCEWithLogitsLoss()
    xent = nn.CrossEntropyLoss()
    mse = nn.MSELoss()
    
    def loss_fct(y_bin_pred, y_cat_pred, y_true):
        
        # bce to be computed on all samples
        loss_bin = bce(y_bin_pred, y_true[:,0])
        
        # cross-entropy only computed on samples for which a category is available (!= -1)
        mask = y_true[:, 1] != -1
        
        loss_xent = xent(y_cat_pred[mask], y_true[mask, 1].long())
        
        # TODO: handle cases where is_consp but no label! they should not be brough to 0 0 0 0
        
        # cases that should be covered by MSE (non-consp + consp where gt is known)
        mask = (y_true[:,0] == 0)|((y_true[:,0] == 1)&(y_true[:,1] != -1))
        gt_cat = torch.zeros((mask.sum(), y_cat_pred.shape[1]), device=y_cat_pred.device)
        # gt_cat[y_true[:,1]!=-1, y_true[y_true[:,1]!=-1,1].long()] = 1.
    
        gt_cat[:, y_true[mask,1].long()] = 1.
        loss_mse = mse(torch.sigmoid(y_bin_pred[mask]).reshape(-1,1) * y_cat_pred[mask], gt_cat)
        
        l_bin = 1.
        l_cat = 10.
        l_mse = 0.
        
        return l_bin * loss_bin + l_cat * torch.nan_to_num(loss_xent) + l_mse * torch.nan_to_num(loss_mse)
    return loss_fct

In [92]:
print("n_tokens", n_tokens)
model = ConvModel(n_tokens).cuda()
opt = optim.Adam(model.parameters())
# loss_fct = nn.BCEWithLogitsLoss()
loss_fct = bce_xent()

n_tokens 15393


In [93]:
from torch.utils.data import DataLoader, TensorDataset

ds = TensorDataset(X_train, y_train)
dl = DataLoader(ds, batch_size=4, shuffle=True)

In [94]:
from sklearn.metrics import roc_curve, f1_score, roc_auc_score

In [95]:
import numpy as np

In [96]:
from tqdm import tqdm

cumul_loss = 0

y_val_bin = y_val.cpu().detach().numpy()[:, 0]
y_val_cat = y_val.cpu().detach().numpy()[:, 1]

for epoch in range(20):
    with tqdm(dl) as bar:
        for X_batch, y_batch in bar:
            opt.zero_grad()
            
            y_pred_bin, y_pred_cat = model(X_batch)
            
            loss = loss_fct(y_pred_bin, y_pred_cat, y_batch)
            loss.backward()
            opt.step()
            
            if np.isnan(loss.item()):
                break
            cumul_loss = cumul_loss * .7 + loss.item() * .3
            bar.set_postfix(loss=cumul_loss)
        
        with torch.no_grad():
            model.eval()
            y_pred_val_bin, y_pred_val_cat  = model(X_val)
            y_pred_val_bin = y_pred_val_bin.cpu().detach().numpy()
            y_pred_val_cat = y_pred_val_cat.cpu().detach().numpy().argmax(axis=1)

            
            fpr, tpr, thresh = roc_curve(y_val_bin, y_pred_val_bin)
            best_thresh = thresh[((1-tpr)**2 + fpr**2).argmin()]
            y_pred_val_bin = y_pred_val_bin > best_thresh
            
            mask = y_val_cat != -1
            
            f1_bin = f1_score(y_val_bin, y_pred_val_bin, average="macro")
            f1_cat = f1_score(y_val_cat[mask], y_pred_val_cat[mask], average="macro")
            
            print(best_thresh,"A",f1_bin, "B", f1_cat, "TOTAL", f1_bin * .6 + f1_cat * .4)
            model.train()

100%|██████████| 401/401 [00:04<00:00, 82.11it/s, loss=9.46] 


0.11657225 A 0.650877149450386 B 0.2892279241793805 TOTAL 0.5062174593419837


100%|██████████| 401/401 [00:03<00:00, 113.42it/s, loss=7.23]


-0.0247553 A 0.6756454816285998 B 0.35978270549470115 TOTAL 0.5493003711750404


100%|██████████| 401/401 [00:03<00:00, 117.65it/s, loss=5.44]


...

0.06403023 A 0.710372793175341 B 0.40376075401391853 TOTAL 0.587727977510772


100%|██████████| 401/401 [00:03<00:00, 118.30it/s, loss=0.228]


0.3759104 A 0.6987794380832657 B 0.4261083743842365 TOTAL 0.589711012603654


100%|██████████| 401/401 [00:03<00:00, 118.20it/s, loss=0.264]


-0.272348 A 0.718996190056362 B 0.4510193069110266 TOTAL 0.6118054367982277


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
import random
random.random()

0.7859071514380385

In [ ]:
y_pred_val_proba

tensor([-0.8548, -1.3396,  0.2361, -0.7347, -1.3120, -0.7089,  0.3607, -1.3095,
        -1.1451,  0.3435,  0.0540, -0.5253,  0.0278, -0.1222,  0.2268,  0.0471,
        -1.0794,  0.0023,  0.5623, -1.2321,  0.0393, -0.3050, -0.7990, -0.5041,
         0.1838,  0.6330, -1.1760, -0.3897,  0.4732, -0.1427, -0.2376, -0.2320,
         0.6403, -0.0478, -1.1678,  0.1183,  0.5759, -0.4073,  0.3987,  0.6902,
        -1.2335,  0.2996,  0.1060, -1.1795, -0.4005, -0.4899,  0.0402, -1.1941,
        -0.3558, -0.3094,  0.2454,  0.5827,  0.3673, -1.3603,  0.2070, -1.2239,
        -1.1214,  0.1465,  0.4019,  0.1645, -0.3064, -1.1178, -0.6988,  0.4551,
        -0.5320, -0.4539,  0.3870,  0.2336,  0.0815,  0.4184,  0.3905,  0.0836,
        -0.1238, -1.3310, -0.7574, -0.2628, -0.3751, -1.2006, -0.0078,  0.4473,
        -0.1176, -0.0770,  0.3524,  0.0364,  0.4665, -0.2952, -0.1799,  0.2434,
         0.4023, -0.0513,  0.6520,  0.4968,  0.1165,  0.4282, -0.3582, -1.3414,
         0.5521,  0.3326,  0.2808,  0.36

In [ ]:
tfidf = TfidfVectorizer()

X = tfidf.fit_transform(df.comment_text)
# y = df.conspiratorial.values
y = df.conspiracy.values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=42)

In [38]:
from sklearn.metrics import classification_report

In [39]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.65      1.00      0.79        90
           1       0.73      0.34      0.46        47
           2       0.00      0.00      0.00        14
           3       1.00      0.18      0.31        11

    accuracy                           0.67       162
   macro avg       0.59      0.38      0.39       162
weighted avg       0.64      0.67      0.59       162



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [160]:
import numpy as np
def compute_metrics(eval_pred):
    (logits, _), labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # predictions = torch.max(logits, axis=1).indices
    return f1.compute(predictions=predictions, references=labels, average="macro")

In [161]:
from datasets import load_dataset, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split


In [162]:
df = pd.read_csv("subtaskB_train.csv", index_col=0)
df.drop(columns=["topic"], inplace=True)
df.columns = ["text", "labels"]
df_train, df_test = train_test_split(df, train_size=.8)

ds_train = Dataset.from_pandas(df_train, split="train")
ds_test = Dataset.from_pandas(df_test, split="test")

In [163]:
def tokenize_func(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=200)

In [164]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("morenolq/bart-it")


In [165]:
ds_train_tok = ds_train.map(tokenize_func, batched=True)
ds_test_tok = ds_test.map(tokenize_func, batched=False)

Map:   0%|          | 0/648 [00:00<?, ? examples/s]

Map:   0%|          | 0/162 [00:00<?, ? examples/s]

In [166]:
set(df.labels)

{0, 1, 2, 3}

In [167]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    num_train_epochs=5,
)

import evaluate

f1 = evaluate.load("f1")

In [168]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel

model = AutoModelForSequenceClassification.from_pretrained("morenolq/bart-it", num_labels=4)
tokenizer = AutoTokenizer.from_pretrained("morenolq/bart-it")
# oppure, dipende da quello che volete fare

Some weights of the model checkpoint at morenolq/bart-it were not used when initializing BartForSequenceClassification: ['lm_head.weight', 'final_logits_bias']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at morenolq/bart-it and are newly initialized: ['classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task t

In [170]:
trainer = Trainer(
    model=model,
    args=training_args, 
    train_dataset=ds_train_tok,
    eval_dataset=ds_test_tok,
    compute_metrics=compute_metrics,
)

In [171]:
def num_params(model):
    return sum([ np.prod(tuple(x.shape)) for x in list(model.parameters()) ])

In [172]:
num_params(model)

141346564

In [173]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,No log,0.500953,0.784207
2,No log,0.468956,0.850243
3,No log,0.681765,0.839796
4,No log,0.670220,0.858380
5,No log,0.697096,0.852897


TrainOutput(global_step=405, training_loss=0.1985308235074267, metrics={'train_runtime': 58.6632, 'train_samples_per_second': 55.231, 'train_steps_per_second': 6.904, 'total_flos': 388157033664000.0, 'train_loss': 0.1985308235074267, 'epoch': 5.0})

## Multi-Task

In [174]:
import torch.nn as nn
import torch

In [175]:
from transformers.utils import ModelOutput

In [176]:
class MultiTaskModel(nn.Module):
    def __init__(self, encoder, max_size=200, n_classes=2):
        super().__init__()
        self.encoder = encoder
        self.fc1 = nn.Linear(768, 768)
        if n_classes == 2:
            self.fc2 = nn.Linear(768, 1)
            self.loss = nn.BCEWithLogitsLoss()
        else:
            self.fc2 = nn.Linear(768, n_classes)
            self.loss = nn.CrossEntropyLoss()

        self.encoder._init_weights(self.fc1)
        self.encoder._init_weights(self.fc2)
    
    def forward(self, input_ids, attention_mask, labels=None):
        x = self.encoder(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        x = x[:, 0] # use only [CLS]
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.fc2(x)
        if labels is not None:
            loss = self.loss(x, labels)
            return ModelOutput({"loss": loss, "logits": x })

        return ModelOutput({ "logits": x })

In [177]:
from transformers.utils import ModelOutput

In [178]:
from transformers import Trainer

class MultiTaskTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss = nn.CrossEntropyLoss()
        # self.loss = nn.BCEWithLogitsLoss()
        
    def compute_loss(self, model, inputs, return_outputs=False):
        # implement custom logic here
        output = model(inputs["input_ids"], inputs["attention_mask"])
        
        loss = self.loss(output.get("logits"), inputs["labels"])
        if return_outputs:
            return loss, output
        return loss


In [179]:
import numpy as np


In [180]:
encoder = AutoModel.from_pretrained("morenolq/bart-it")#, output_hidden_states=True)
model = MultiTaskModel(encoder=encoder, n_classes=4)

Some weights of the model checkpoint at morenolq/bart-it were not used when initializing BartModel: ['lm_head.weight', 'final_logits_bias']
- This IS expected if you are initializing BartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [181]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    num_train_epochs=10,
)

import evaluate

f1 = evaluate.load("f1")

import numpy as np
def compute_metrics(eval_pred):
    logits = eval_pred.predictions
    labels = eval_pred.label_ids
    # (logits, _), labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # predictions = torch.max(logits, axis=1).indices
    return f1.compute(predictions=predictions, references=labels, average="macro")

trainer = Trainer(
# trainer = MultiTaskTrainer(
    model=model,
    args=training_args, 
    train_dataset=ds_train_tok,
    eval_dataset=ds_test_tok,
    compute_metrics=compute_metrics,
)

In [182]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,No log,0.540235,0.782136
2,No log,0.563055,0.800301
3,No log,0.708294,0.831806
4,No log,0.801645,0.867346
5,No log,0.869802,0.862512
6,No log,0.905962,0.872841
7,0.204000,0.921589,0.872841
8,0.204000,0.931924,0.871923
9,0.204000,0.937373,0.871923
10,0.204000,0.939788,0.871923


TrainOutput(global_step=810, training_loss=0.1260279884170971, metrics={'train_runtime': 118.3858, 'train_samples_per_second': 54.736, 'train_steps_per_second': 6.842, 'total_flos': 0.0, 'train_loss': 0.1260279884170971, 'epoch': 10.0})